# Eat Safe, Love

## Notebook Set Up

In [7]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [8]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [9]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [11]:
# review the collections in our database
print(db.list_collection_names())


['establishments']


In [15]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [17]:
# Find the establishments with a hygiene score of 20
query = {"hygiene_score": 20}

# Use count_documents to display the number of documents in the result
count = db.establishments.count_documents(query)
print(f"Number of establishments with hygiene score equal to 20: {count}")

# Display the first document in the results using pprint
result = db.establishments.find_one(query)
print("First document with hygiene score equal to 20:")
pprint(result)



Number of establishments with hygiene score equal to 20: 0
First document with hygiene score equal to 20:
None


In [ ]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(list(db.establishments.find(query)))

# Display the number of rows in the DataFrame
print("Number of rows in the DataFrame:", len(df))

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df.head(10))

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {"LocalAuthorityName": "London", "RatingValue": {"$gte": 4}}

# Use count_documents to display the number of documents in the result
count = db.establishments.count_documents(query)
print(f"Number of establishments in London with RatingValue >= 4: {count}")

# Display the first document in the results using pprint
result = db.establishments.find_one(query)
print("First document in London with RatingValue >= 4:")
pprint(result)

In [ ]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(list(db.establishments.find(query)))

# Display the number of rows in the DataFrame
print("Number of rows in the DataFrame:", len(df))

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df.head(10))

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score


degree_search = 0.01
latitude = 0.03
longitude = 0.11


query = {
    "RatingValue": 5,
    "Latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search},
    "Longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search}
}


sort = [("HygieneScore", 1)]  


limit = 5

# Print
results = db.establishments.find(query).sort(sort).limit(limit)
for result in results:
    pprint(result)


In [ ]:
# Convert result to Pandas DataFrame
df = pd.DataFrame(list(results))

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

total_establishments = 0
for result in results:
    total_establishments += result["count"]
    print(f"Number of establishments with hygiene score 0 in {result['_id']}: {result['count']}")

print(f"Total number of establishments with hygiene score 0: {total_establishments}")

# Print the number of documents in the result
total_establishments = 0
for result in results:
    total_establishments += result["count"]
    print(f"Number of establishments with hygiene score 0 in {result['_id']}: {result['count']}")

print(f"Total number of establishments with hygiene score 0: {total_establishments}")

# Print the first 10 results
results = db.establishments.aggregate(pipeline)
print("First 10 results:")
for i, result in enumerate(results):
    print(result)
    if i == 9:
        break





In [ ]:
# Convert the result to a Pandas DataFrame
# Display the number of rows in the DataFrame
# Display the first 10 rows of the DataFrame

pipeline = [
    {"$match": {"HygieneScore": 0}},  # Match establishments with a hygiene score of 0
    {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}},  # Group by Local Authority and count establishments
    {"$sort": {"count": -1}}  # Sort by count in descending order
]

df = pd.DataFrame(list(db.establishments.aggregate(pipeline)))

print("Number of rows in the DataFrame:", len(df))

print("First 10 rows of the DataFrame:")
print(df.head(10))
